# Approaching (Almost) Any NLP Problem on Kaggle

이 코드는 kaggle의 https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle 코드를 가져와 리뷰한 것입니다.

In this post I'll talk about approaching natural language processing problems on Kaggle. As an example, we will use the data from this competition. We will create a very basic first model first and then improve it using different other features. We will also see how deep neural networks can be used and end this post with some ideas about ensembling in general.

### This covers:
- tfidf 
- count features
- logistic regression
- naive bayes
- svm
- xgboost
- grid search
- word vectors
- LSTM
- GRU
- Ensembling

*NOTE*: This notebook is not meant for achieving a very high score on the Leaderboard for this dataset. However, if you follow it properly, you can get a very high score with some tuning. ;)

So, without wasting any time, let's start with importing some important python modules that I'll be using.

In [52]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import warnings
warnings.filterwarnings(action='ignore')

Let's load the datasets

In [53]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

A quick look at the data

In [54]:
train.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [55]:
test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


The problem requires us to predict the author, i.e. EAP, HPL and MWS given the text. In simpler words, text classification with 3 different classes.

For this particular problem, Kaggle has specified multi-class log-loss as evaluation metric. This is implemented in the follow way (taken from: https://github.com/dnouri/nolearn/blob/master/nolearn/lasagne/util.py)

In [56]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

https://nolearndocs.readthedocs.io/en/latest/_modules/nolearn/metrics.html
    
- nolearn -> scikit-learn과 연동되며 기계학습에 유용한 여러 함수를 담고 있음.

- 이해를 위해, 아래에서 함수가 적용된 multiclass_logloss(yvalid, predictions)를 살펴보면,
yvalid.shape[0]가 1958이고, predictions.shape[1]이 3이므로, np.zeros((1958, 3))는 0으로 채워진 1958행 3열의 matrix를 반환하는 것이다. 

- yvalid: array, shape = [n_samples]

- predictions: array, shape = [n_samples, n_classes]

- enumerate 함수는 인덱스와 값이 같이 출력되는 함수로, 0으로 채워진 matrix에서 행-인덱스, 열-값에 해당하는 부분을 1로 바꿔준다.

- np.clip(배열, 최소값 기준, 최대값 기준): 최소값과 최대값 조건으로 범위 기준을 벗어나는 값은 일괄적으로 최소값, 최대값으로 대치해준다.

- le-15 = 0.000 000 000 000 001, 예측된 확률이 0인 경우, log loss값이 무한대가 된다. 그래서 실제 예측 확률은 0이라도, 15승분의 1의 값을 넣어두어 손실을 막는다. 

<img src = "https://miro.medium.com/max/1162/1*bUv2Dgcfw6OG9vhcpRXIeg.png">

In [57]:
# 일반적인 logloss 계산 코드

def logloss(true_label, predicted, eps=1e-15):
    p = np.clip(predicted, eps, 1 - eps)
    if true_label == 1:
        return -log(p)
    else:
        return -log(1 - p)

In [58]:
# Binary clssification 계산 코드

def logloss(true_label, predicted_prob):
    if true_label == 1:
        return -log(predicted_prob)
    else:
        return -log(1 - predicted_prob)

<img src = "https://miro.medium.com/max/1096/1*rdBw0E-My8Gu3f_BOB6GMA.png">

We use the LabelEncoder from scikit-learn to convert text labels to integers, 0, 1 2

- 이 데이터가 Edgar Allan Poe /  HP Lovecraft / Mary Shelley 3명의 작가를 예측하는 문제이므로, 문자를 수치화해주는  LabelEncoder를 사용

In [59]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(train.author.values)

In [60]:
print(lbl_enc.classes_)

['EAP' 'HPL' 'MWS']


In [61]:
print(train.author.values, "==>", y)

['EAP' 'HPL' 'EAP' ... 'EAP' 'EAP' 'HPL'] ==> [0 1 0 ... 0 0 1]


Before going further it is important that we split the data into training and validation sets. We can do it using `train_test_split` from the `model_selection` module of scikit-learn.

In [62]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

- arrays : 분할시킬 데이터를 입력 (Python list, Numpy array, Pandas dataframe 등..)

- test_size : 테스트 데이터셋의 비율(float)이나 갯수(int) (default = 0.25)

- train_size : 학습 데이터셋의 비율(float)이나 갯수(int) (default = test_size의 나머지)

- random_state : 데이터 분할시 셔플이 이루어지는데 이를 위한 시드값 (int나 RandomState로 입력)

- shuffle : 셔플여부설정 (default = True)

- stratify : 지정한 Data의 비율을 유지한다. 예를 들어, Label Set인 Y가 25%의 0과 75%의 1로 이루어진 Binary Set일 때, stratify=Y로 설정하면 나누어진 데이터셋들도 0과 1을 각각 25%, 75%로 유지한 채 분할된다.

In [63]:
print (xtrain.shape)
print (xvalid.shape)

(17621,)
(1958,)


## Building Basic Models

Let's start building our very first model. 

Our very first model is a simple TF-IDF (Term Frequency - Inverse Document Frequency) followed by a simple Logistic Regression.

### 1) TfidfVectorizer
TF-IDF는 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어의 경우 문서 구별 능력이 떨어진다고 보아 가중치를 축소하는 방법

단어 빈도 또는 등장 여부를 그대로 임베딩으로 쓰는 것에는 단점이 있음. 어떤 문서에든 쓰여서 해당 단어가 많이 나타났다 하더라도 문서의 주제를 가늠하기 어려운 경우가 있기 때문. Tfidf는 이러한 단점을 보완한다.

- TF: 어떤 단어가 특정 문서에서 얼마나 많이 쓰였는지
- DF: 특정 단어가 나타난 문서의 수
- IDF: 전체 문서 수를 해당 단어의 DF로 나눈뒤 로그를 취한 값, 그 값이 클수록 특이한 단어

<img src = "https://t1.daumcdn.net/cfile/tistory/22346248538D3D1205">

In [64]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1,
            stop_words = 'english')

# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

- mid-df : DF(document-frequency, 문서의 수)의 최소 빈도값을 설정, 해당 값보다 작은 DF를 가진 단어들은 단어사전(vocabulary_)에서 제외하고, 인덱스를 부여하지 않음

- max_features: 최대 feature를 설정, 다른 데이터의 단어들은 10 정도를 가지는데, 어떤 데이터만 단어 종류가 100이 넘어간다고 하면, 이 100에 맞추어 feature의 수가 엄청 늘어나게 된다. 이 경우, 모델 성능이 저하될 수도 있다.

- strip_accents : 문자 정규화, 'ascii', 'unicode'가 있는데, 'unicode'가 느리지만, 모든 문자에 적용 가능한 방법이다. default = None

- analyzer : 학습단위를 결정, 'word'라고 설정시 학습 단위를 단어로 설정하고, 'char'로 설정시 학습 단위를 글자로 설정

- token_pattern : 토큰을 구성하는 정규표현식 정의

- ngram_range : 단어 묶음 범위를 결정, (1, 1)이라면 단어의 묶음을 1개부터 1개까지 설정하라는 뜻으로, 기존과 차이가 없다. (1, 2)라면, 단어의 묶음을 1개부터 2개까지 설정하라는 뜻으로, 단어 사전에는 1개 단어 묶음과 2개 단어 묶음이 모두 존재하게 된다. (very good)

- use_idf : inverse-document-frequency(특정한 단어가 들어있는 문서의 수에 반비례하는 수) 활용 여부를 결정, default = True

- smooth_idf : idf 가중치의 스무딩(smoothing) 여부를 결정

- sublinear_tf : TF(Term-frequency, 단어 빈도)의 스무딩(smooothing) 여부를 결정, True/False

- stop_words : 불용어 설정, default = None

-> 불용어 제거 : 갖고 있는 데이터에서 유의미한 단어 토큰만을 선별하기 위해서는 큰 의미가 없는 단어 토큰을 제거하는 작업, I, my, me, over, 조사, 접미사 같은 단어들은 문장에서는 자주 등장하지만 실제 의미 분석을 하는데는 거의 기여하는 바가 없는 경우가 많다.

In [65]:
def sublinear_func(input): 
    rst = 1 + np.log(input) 
    return rst 

print(sublinear_func(100)) 
print(sublinear_func(10000))

5.605170185988092
10.210340371976184


- 결과를 보면 100은 5.6 정도로, 10000은 10.2 정도로 값이 확 줄어든 것을 확인할 수 있다.
이처럼 sublinear_tf는 높은 TF값을 완만하게 처리하는 효과를 가지고 있다.
TF의 아웃라이어가 너무 심한 데이터의 경우, 이 파라미터를 True로 바꿔주면 어느정도 효과를 기대할 수 있다


In [66]:
# Fitting a simple Logistic Regression on TFIDF
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.572 


And there we go. We have our first model with a multiclass logloss of 0.626.

But we are greedy and want a better score. Lets look at the same model with a different data.

Instead of using TF-IDF, we can also use word counts as features. This can be done easily using CountVectorizer from scikit-learn.

### 2) CountVectorizer

1. 문서를 토큰 리스트로 변환한다.
2. 각 문서에서 토큰의 출현 빈도를 센다.
3. 각 문서를 BOW 인코딩 벡터로 변환한다.

<img src = "https://www.educative.io/api/edpresso/shot/5197621598617600/image/6596233398321152">

In [67]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

- stop_words : stop words 목록.‘english’이면 영어 불용어 목록 사용, default = None

- analyzer : 학습 단위를 결정, {‘word’, ‘char’, ‘char_wb’} - {단어 n-그램, 문자 n-그램, 단어 내의 문자 n-그램}, 혹은 함수

- token_pattern : 토큰을 구성하는 정규표현식 정의

- tokenizer : 토큰 생성 함수, default = None

- ngram_range : (min_n, max_n) 튜플, n-그램 범위

- max_df : 정수 또는 [0.0, 1.0] 사이의 실수. 단어 사전에 포함되기 위한 최대 빈도, default = 1

- min_df : 정수 또는 [0.0, 1.0] 사이의 실수. 단어 사전에 포함되기 위한 최소 빈도, default = 1

In [68]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.527 


Aaaaanddddddd Wallah! We just improved our first model by 0.1!!!

Next, let's try a very simple model which was quite famous in ancient times - Naive Bayes.

Let's see what happens when we use naive bayes on these two datasets:

In [69]:
# Fitting a simple Naive Bayes on TFIDF
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.578 


Good performance! But the logistic regression on counts is still better! What happens when we use this model on counts data instead?

In [70]:
# Fitting a simple Naive Bayes on Counts
clf = MultinomialNB()
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.485 


Whoa! Seems like old stuff still works good!!!! One more ancient algorithms in the list is SVMs. Some people "love" SVMs. So, we must try SVM on this dataset.

Since SVMs take a lot of time, we will reduce the number of features from the TF-IDF using Singular Value Decomposition before applying SVM. 

Also, note that before applying SVMs, we *must* standardize the data.

In [71]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

<img src = "https://wikidocs.net/images/page/24949/svd%EC%99%80truncatedsvd.PNG">

- atent Semantic Indexing -> 특이값 분해 계산: 차원 축소

동의어나 다의어로부터 오는 문제를 해결하기 위해 정보 검색 분야에 널리 사용되고 있다. 

TruncatedSVD를 통해 데이터를 중요순서대로 끊어내는 프로세스를 할 수 있다.
절단된 SVD는 대각 행렬 Σ의 대각 원소의 값 중에서 상위값 t개만 남게 된다. t는 우리가 찾고자하는 토픽의 수를 반영한 하이퍼파라미터 값이다. t를 선택하는 것은 쉽지 않다. t를 크게 잡으면 기존의 행렬 A로부터 다양한 의미를 가져갈 수 있지만, t를 작게 잡아야만 노이즈를 제거할 수 있기 때문이다.

자연어 처리 분야에서는 위와 같은 과정을 통해, 설명력이 낮은 정보를 삭제하고 설명력이 높은 정보를 남길 수 있다.

한편, SVM은 scaling에 민감하기 때문에, 평균을 0, 표준표차가 1이 되도록 하는 standardscaling을 진행한다.

Now it's time to apply SVM. After running the following cell, feel free to go for a walk or talk to your girlfriend/boyfriend. :P

In [72]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.740 


Oops! time to get up! Looks like SVM doesn't perform well on this data...! 

Before moving further, lets apply the most popular algorithm on Kaggle: xgboost!

In [73]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.782 


In [74]:
# Fitting a simple xgboost on tf-idf
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.773 


In [75]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.783 


In [76]:
# Fitting a simple xgboost on tf-idf svd features
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.789 


- XGBoost(eXtra Gradient Boost): 트리 기반의 알고리즘의 앙상블 학습에서 각광받는 알고리즘 
    
- max_depth : 트리 기반 알고리즘의 max_depth와 동일, 0을 지정하면 깊이의 제한이 없음, 너무 크면 과적합(통상 3~10정도 적용)

- n_estimators : 생성할 weak learner의 수

- colsample_bytree : GBM(그레디언트 부스트)의 max_features와 유사, 트리 생성에 필요한 피처의 샘플링에 사용, 피처가 많을 때 과적합 조절에 사용, 범위: 0 ~ 1

- subsample : GBM의 subsample과 동일, 데이터 샘플링 비율 지정(과적합 제어), 일반적으로 0.5~1 사이의 값을 사용, 범위: 0 ~ 1

- nthread : CPU 실행 스레드 개수 조정, Default는 전체 다 사용하는 것, 멀티코어/스레드 CPU 시스템에서 일부CPU만 사용할 때 변경

- learning_rate : 학습률 결정, 범위 0~1


트리 모델의 중요 매개변수는 트리의 개수를 지정하는 n_estimators와 이전 트리의 오차를 보정하는 정도를 조절하는 learning_rate이다. 이 두 매개변수는 매우 깊게 연관되며 learning_rate를 낮추면 비슷한 복잡도의 모델을 만들기 위해서 더 많은 트리를 추가해야 한다. n_estimators가 클수록 좋은 랜덤 포레스트와는 달리 그래디언트 부스팅에서 n_estimators를 크게 하면 모델이 복잡해지고 과대적합될 가능성이 높아진다. 일반적인 관례는 가용한 시간과 메모리 한도에서 n_estimators를 맞추고 나서 적절한 learning_rate를 찾는 것이다.

Seems like no luck with XGBoost! But that is not correct. I haven't done any hyperparameter optimizations yet. And since I'm lazy, I'll just tell you how to do it and you can do it on your own! ;). This will be discussed in the next section:


## Grid Search

Its a technique for hyperparameter optimization. Not so effective but can give good results if you know the grid you want to use. I specify the parameters that should usually be used in this post: http://blog.kaggle.com/2016/07/21/approaching-almost-any-machine-learning-problem-abhishek-thakur/ Please keep in mind that these are the parameters I usually use. There are many other methods of hyperparameter optimization which may or may not be as effective.

In this section, I'll talk about grid search using logistic regression. 

Before starting with grid search we need to create a scoring function. This is accomplished using the `make_scorer` function of scikit-learn.


- 그리드 서치는 관심 있는 매개변수들을 대상으로 가능한 모든 조합들을 시도하여 초적의 매개변수를 찾는 방법

In [77]:
mll_scorer = metrics.make_scorer(multiclass_logloss, greater_is_better=False, needs_proba=True)

Next we need a pipeline. For demonstration here, i'll be using a pipeline consisting of SVD, scaling and then logistic regression. Its better to understand with more modules in pipeline than just one ;)

- sklearn에서는 custom metric으로 모델을 선정할 수 있다.

- greater_is_better : score function이 높으면 높을 수록 좋음, 만약 손실 함수일 경우 낮음이 좋음.

- needs_proba : 분류 모델에서 predict_proba를 요구하는지 여부

In [78]:
# Initialize SVD
svd = TruncatedSVD()
    
# Initialize the standard scaler 
scl = preprocessing.StandardScaler()

# We will use logistic regression here..
lr_model = LogisticRegression()

# Create the pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

Next we need a grid of parameters:

In [79]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

So, for SVD we evaluate 120 and 180 components and for logistic regression we evaluate three different values of C with l1 and l2 penalty. We can now start grid search on these parameters.

In [80]:
# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done   2 out of  24 | elapsed:    3.3s remaining:   36.7s
[Parallel(n_jobs=-1)]: Done   5 out of  24 | elapsed:    4.4s remaining:   16.5s
[Parallel(n_jobs=-1)]: Done   8 out of  24 | elapsed:    4.6s remaining:    9.3s
[Parallel(n_jobs=-1)]: Done  11 out of  24 | elapsed:    5.3s remaining:    6.3s
[Parallel(n_jobs=-1)]: Done  14 out of  24 | elapsed:    6.0s remaining:    4.3s
[Parallel(n_jobs=-1)]: Done  17 out of  24 | elapsed:    6.7s remaining:    2.7s
[Parallel(n_jobs=-1)]: Done  20 out of  24 | elapsed:    7.4s remaining:    1.5s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    8.7s finished


Best score: -0.740
Best parameters set:
	lr__C: 0.1
	lr__penalty: 'l2'
	svd__n_components: 180


- param_grid : 찾고자 하는 파라미터, dict 형식

- verbose: 상세 정보를 보여주는 정도를 선택 가능, 높을수록 더 많은 메시지가 표시

- n_jobs : 벙렬처리 갯수, default는 1, 이 값을 증가시키면 내부적으로 멀티 프로세스를 사용하여 그리드서치를 수행. 만약 CPU 코어의 수가 충분하다면 n_jobs를 늘릴 수록 속도가 증가

- iid : True로 설정 시, data가 전체에 동일하게 배포되는 것으로 간주, loss는 샘플 당 전체 loss를 감소시킴

- refit : default가 True, 좋은 estimator로 수정되어짐

The score comes similar to what we had for SVM. This technique can be used to finetune xgboost or even multinomial naive bayes as below. We will use the tfidf data here:

In [81]:
nb_model = MultinomialNB()

# Create the pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# parameter grid
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# Initialize Grid Search Model
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# Fit Grid Search Model
model.fit(xtrain_tfv, ytrain)  # we can use the full data here but im only using xtrain. 
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits
Best score: -0.492
Best parameters set:
	nb__alpha: 0.1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0256s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    0.1s finished


This is an improvement of 8% over the original naive bayes score!

In NLP problems, it's customary to look at word vectors. Word vectors give a lot of insights about the data. Let's dive into that.

## Word Vectors

Without going into too much details, I would explain how to create sentence vectors and how can we use them to create a machine learning model on top of it. I am a fan of GloVe vectors, word2vec and fasttext. In this post, I'll be using the GloVe vectors. You can download the GloVe vectors from here `http://www-nlp.stanford.edu/data/glove.840B.300d.zip`

- glove: word2vec의 단점을 해결하기 위해, 미국 스탠포드 대학에서 개발한 방법론

- word2vec은 실제값과 예측값에 대한 오차를 손실 함수를 통해 줄여나가며 학습하는 예측 기반의 방법론이다. 중심 단어로 주변 단어, 주변 단어로 중심 단어를 예측하는 과정에서 단어를 벡터화하는 것으로, 임베딩된 단어의 내적이 코사인 유사도가 되도록 한다. 

- 연구진은 단어의 공동 출현 횟수를 세어서 정방행렬 형태로 기록한 후, 그 행렬에 SVD를 적용해 2개의 가중치 행렬을 얻었다. glove는 임베딩된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 한다. 이 기법은 말뭉치 전체에 대한 동시 등장 확률 global vector를 최적화하는 방법론이다. 이를 통해, word2vec의 은닉 가중치 행렬과 출력 가중치 행렬을 훨씬 짧은 시간으로 산출할 수 있다.

- "임베딩된 단어 벡터 간 유사도 측정을 수월하게 하면서도 말뭉치 전체의 통계 정보를 좀 더 잘 반영해보자"가 glove의 목표이다.

- word2vec과 glove 모두 핵심은 단순히 단어를 'count'하는 방식이 아니라 단어의 '동시 등장 여부'를 확률 기반으로 예측한다는 것

In [ ]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

In [83]:
# this function creates a normalized vector for the whole sentence
def sent2vec(s):
    words = str(s).lower().encode().decode('utf-8')
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

- isalpha()는 문자열이 문자인지 아닌지를 True, Flase로 리턴해준다.

In [84]:
# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_glove = [sent2vec(x) for x in tqdm(xvalid)]


100%|██████████| 17621/17621 [00:06<00:00, 2569.66it/s]

100%|██████████| 1958/1958 [00:00<00:00, 2267.60it/s]


In [85]:
xtrain_glove = np.array(xtrain_glove)
xvalid_glove = np.array(xvalid_glove)

Let's see the performance of xgboost on glove features:

In [86]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.791 


In [87]:
# Fitting a simple xgboost on glove features
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_glove)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.747 


we see that a simple tuning of parameters can improve xgboost score on GloVe features! Believe me you can squeeze a lot more from it.

## Deep Learning

But this is an era of deep learning! We cant live without training a few neural networks. Here, we will train LSTM and a simple dense network on the GloVe features. Let's start with the dense network first:

In [88]:
# scale the data before any neural net:
scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xvalid_glove_scl = scl.transform(xvalid_glove)

In [89]:
# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(ytrain)
yvalid_enc = np_utils.to_categorical(yvalid)

- keras.np_utils.categorical()을 사용하여 원핫인코딩(One-Hot-Encoding)로 변환

In [90]:
ytrain_enc[:5]

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [91]:
# create a simple 3 layer sequential neural net
model = Sequential()

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Dense(3))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

- Dropout: 특정 확률로 node의 activation을 지움. 과적합 방지

- 배치 정규화 (BatchNormalization): 경사하강법에서 Gradient Vanishing / Gradient Exploding 이 일어나지 않도록 함, 입력값을 평균 0, 분산 1로 정규화해 네트워크의 학습이 잘 일어나도록 돕는 방식

- activation: 'softmax'는 입력받은 값을 출력으로 0~1 사이 값으로 정규화하여 총합은 항상 1이 되도록 하는 함수, 분류하고 싶은 클래스의 수만큼 출력으로 구성되고, 가장 큰 출력 값을 부여받은 클래스가 확률이 가장 높은 것으로 이용된다.

- loss: categorical_crossentropy - Softmax activation 뒤에 Cross-Entropy loss를 붙인 형태로 주로 사용하기 때문에 Softmax loss 라고도 불린다. multi-class clssification 문제에서 주로 쓰인다. 

- optimizer: adam - RMRProp + Momentum

<img src = "https://icim.nims.re.kr/file/70679cf085a049679f03736d7afad264.png">

<img src = "https://image.slidesharecdn.com/random-170910154045/95/-49-638.jpg?cb=1505089848">

In [92]:
model.fit(xtrain_glove_scl, y=ytrain_enc, batch_size=64, 
          epochs=5, verbose=1, 
          validation_data=(xvalid_glove_scl, yvalid_enc))

Epoch 1/5
264/276 [===========================>..] - ETA: 0s - loss: 0.9686

  5%|▌         | 892/17621 [52:28<16:24:12,  3.53s/it]


276/276 [==============================] - 1s 5ms/step - loss: 0.9628 - val_loss: 0.7687
Epoch 2/5
276/276 [==============================] - 1s 4ms/step - loss: 0.7499 - val_loss: 0.7318
Epoch 3/5
276/276 [==============================] - 1s 4ms/step - loss: 0.6872 - val_loss: 0.7180
Epoch 4/5
276/276 [==============================] - 1s 4ms/step - loss: 0.6533 - val_loss: 0.7248
Epoch 5/5
276/276 [==============================] - 1s 3ms/step - loss: 0.6170 - val_loss: 0.7112


You need to keep on tuning the parameters of the neural network, add more layers, increase dropout to get better results. Here, I'm just showing that its fast to implement and run and gets better result than xgboost without any optimization :)

To move further, i.e. with LSTMs we need to tokenize the text data

- rnn은 다른 딥러닝 모델처럼 오차를 줄이기 위해 입력의 반대 방향으로 back propagation을 한다. 이 때 앞에서 들어온 시퀀스 정보를 뒤로 넘겨주변서 비선형 함수, 즉 탄젠트 함수를 지나고 이에 따라 vanishing gradient 문제가 생기게 된다. 

- 이 문제를 해결하기 위해 나온 것이 LSTM 모델, Long Short Term Memory라는 이름처럼 장기간, 단기간 모두에 적용 가능하다. 

In [93]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 70

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

- 토큰화: sentence를 토큰화, num_words에 숫자 입력시 빈도 숫자가 높은 num_words만큼의 단어만을 활용한다.

- sequence로 변환: sentences를 Tokenizer를 통해 기계가 알아들을 수 있는 numerical value로 변환한다.

- padding: sequences는 길이가 들쭉날쭉하므로, 길이를 맞춰줘야 함. 길이가 긴 문장을 자르거나 짧은 문장은 padding(0으로 채움) 처리해준다.

In [94]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 25943/25943 [00:00<00:00, 296675.42it/s]


전체 단어를 300차원의 Vector로 임베딩하고 싶은 경우엔 단어 개수 x 300의 Embedding Matrix가 필요함

1 x 단어 개수와 단어 개수 x 300를 행렬곱하면 자신의 Index로 인덱싱

In [95]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

- SpatialDropout1D:  특정 컬럼을 전체를 떨구는것으로써, 텍스트분석등은 대충 떨궈놓으면 어차피 주위 단어로 유추가 전부 가능하기 때문에 드랍아웃을 하는 의의가 없어지기 때문에 한줄을 죄다 없애버린다.

- activation: CIFAR에서 지원한 hinton 교수가 2006년에 Neural Network가 발전하지 못했던 이유를 연구하다가 새로운 방법을 찾아냈다. 그 동안은 non-linearity에 대해 잘못된 방법을 사용했다는 것이었다. 여기서 non-linearity는 sigmoid 함수를 말한다. vanishing gradient 문제의 발생 원인은 sigmoid 함수에 있었다. sigmoid 함수가 값을 변형하면서 이런 문제가 생긴 것이었다. ReLU 함수는 그림에 있는 것처럼 0보다 작을 때는 0을 사용하고, 0보다 큰 값에 대해서는 해당 값을 그대로 사용하는 방법이다. 음수에 대해서는 값이 바뀌지만, 양수에 대해서는 값을 바꾸지 않는다.

<img src = "https://t1.daumcdn.net/cfile/tistory/22293C50579F7BBF13">

In [96]:
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, verbose=1, validation_data=(xvalid_pad, yvalid_enc))

Epoch 1/100
35/35 [==============================] - 9s 266ms/step - loss: 1.0647 - val_loss: 0.9315
Epoch 2/100
35/35 [==============================] - 9s 256ms/step - loss: 0.9106 - val_loss: 0.7853
Epoch 3/100
35/35 [==============================] - 9s 266ms/step - loss: 0.8456 - val_loss: 0.7538
Epoch 4/100
35/35 [==============================] - 9s 259ms/step - loss: 0.8124 - val_loss: 0.7350
Epoch 5/100
35/35 [==============================] - 9s 259ms/step - loss: 0.7942 - val_loss: 0.7107
Epoch 6/100
35/35 [==============================] - 9s 262ms/step - loss: 0.7706 - val_loss: 0.7074
Epoch 7/100
35/35 [==============================] - 9s 263ms/step - loss: 0.7568 - val_loss: 0.6873
Epoch 8/100
35/35 [==============================] - 9s 259ms/step - loss: 0.7378 - val_loss: 0.6889
Epoch 9/100
35/35 [==============================] - 9s 258ms/step - loss: 0.7278 - val_loss: 0.6707
Epoch 10/100
35/35 [==============================] - 9s 255ms/step - loss: 0.7063 - val_lo

We see that the score is now less than 0.5. I ran it for many epochs without stopping at the best but you can use early stopping to stop at the best iteration. How do I use early stopping?

well, pretty easy. let's compile the model again:

In [97]:
# A simple LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 9s 266ms/step - loss: 1.0340 - val_loss: 0.8331
Epoch 2/100
35/35 [==============================] - 9s 264ms/step - loss: 0.8837 - val_loss: 0.7624
Epoch 3/100
35/35 [==============================] - 9s 255ms/step - loss: 0.8447 - val_loss: 0.7550
Epoch 4/100
35/35 [==============================] - 9s 247ms/step - loss: 0.8185 - val_loss: 0.7486
Epoch 5/100
35/35 [==============================] - 9s 245ms/step - loss: 0.7929 - val_loss: 0.7234
Epoch 6/100
35/35 [==============================] - 8s 236ms/step - loss: 0.7650 - val_loss: 0.6992
Epoch 7/100
35/35 [==============================] - 9s 251ms/step - loss: 0.7514 - val_loss: 0.6884
Epoch 8/100
35/35 [==============================] - 9s 250ms/step - loss: 0.7380 - val_loss: 0.6898
Epoch 9/100
35/35 [==============================] - 9s 260ms/step - loss: 0.7250 - val_loss: 0.6991
Epoch 10/100
35/35 [==============================] - 8s 240ms/step - loss: 0.6979 - val_lo

- earlystop 

- mode : mode 의 default는 auto 인데, 이는 keras 에서 알아서 min, max 여부를 선택하게 된다. performance measure를 정의하고, 이것을 최대화 할지, 최소화 할지를 지정하는 것이다. 그러면 keras 에서 알아서 적절한 epoch 에서 training 을 멈춘다. 
   - auto : 관찰하는 이름에 따라 자동으로 지정
   - min : 관찰하고 있는 항목이 감소되는 것을 멈출 때 종료
   - max : 관찰하고 있는 항목이 증가되는 것을 멈출 때 종료


- min_delta: 개선되고 있다고 판단하기 위한 최소 변화량을 나타낸다. 만약 변화량이 min_delta보다 적은 경우에는 개선이 없다고 판단한다.

- verbose=1 로 지정하면, 언제 keras 에서 training 을 멈추었는지를 화면에 출력할 수 있다.

- patience: 성능이 증가하지 않는다고, 그 순간 바로 멈추는 것은 효과적이지않을 수 있다. patience 는 성능이 증가하지 않는 epoch 을 몇 번이나 허용할 것인가를 정의한다. partience 는 다소 주관적인 기준이다. 사용한 데이터와 모델의 설계에 따라 최적의 값이 바뀔 수 있다. 

One question could be: why do i use so much dropout? Well, fit the model with no or little dropout and you will that it starts to overfit :)

Let's see if Bi-directional LSTM can give us better results. Its a piece of cake to do it with Keras :)

- LSTM은 이전 step이 이후 step에 영향을 주는 것은 고려하지만, 이후 step이 이전 step에 영향을 주는 것은 고려하지 못한다. 따라서 텍스트 데이터는 정방향(시점을 기준으로 과거에서 미래 방향) 추론과 역방향(시점을 기준으로 미래에서 과거 방향) 추론을 모두 고려할 때 비로소 유의미한 결과를 낼 수 있다. 이에 Bi LSTM은 forward와 backward를 병합해 사용하는 방식을 선택하고 있다. 

In [98]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 17s 484ms/step - loss: 1.0450 - val_loss: 0.8760
Epoch 2/100
35/35 [==============================] - 17s 492ms/step - loss: 0.8808 - val_loss: 0.7824
Epoch 3/100
35/35 [==============================] - 17s 478ms/step - loss: 0.8370 - val_loss: 0.7438
Epoch 4/100
35/35 [==============================] - 17s 484ms/step - loss: 0.8067 - val_loss: 0.7452
Epoch 5/100
35/35 [==============================] - 17s 483ms/step - loss: 0.7876 - val_loss: 0.7189
Epoch 6/100
35/35 [==============================] - 17s 489ms/step - loss: 0.7645 - val_loss: 0.7127
Epoch 7/100
35/35 [==============================] - 17s 488ms/step - loss: 0.7535 - val_loss: 0.6970
Epoch 8/100
35/35 [==============================] - 16s 471ms/step - loss: 0.7268 - val_loss: 0.6833
Epoch 9/100
35/35 [==============================] - 17s 483ms/step - loss: 0.7041 - val_loss: 0.6867
Epoch 10/100
35/35 [==============================] - 17s 487ms/step - loss: 0.680

Pretty close! Lets try two layers of GRU:

- GRU(Gated Recurrent Unit)는 2014년 뉴욕대학교 조경현 교수님이 집필한 논문에서 제안되었다. GRU는 LSTM의 장기 의존성 문제에 대한 해결책을 유지하면서, 은닉 상태를 업데이트하는 계산을 줄였다. 다시 말해서, GRU는 성능은 LSTM과 유사하면서 복잡했던 LSTM의 구조를 간단화 시킨 것이다.

In [99]:
# GRU with glove embeddings and two dense layers
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))
model.add(SpatialDropout1D(0.3))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model.fit(xtrain_pad, y=ytrain_enc, batch_size=512, epochs=100, 
          verbose=1, validation_data=(xvalid_pad, yvalid_enc), callbacks=[earlystop])

Epoch 1/100
35/35 [==============================] - 18s 507ms/step - loss: 1.0657 - val_loss: 0.9584
Epoch 2/100
35/35 [==============================] - 16s 469ms/step - loss: 0.9325 - val_loss: 0.9260
Epoch 3/100
35/35 [==============================] - 17s 474ms/step - loss: 0.8702 - val_loss: 0.8150
Epoch 4/100
35/35 [==============================] - 16s 470ms/step - loss: 0.8301 - val_loss: 0.7615
Epoch 5/100
35/35 [==============================] - 16s 470ms/step - loss: 0.8051 - val_loss: 0.7486
Epoch 6/100
35/35 [==============================] - 16s 460ms/step - loss: 0.7797 - val_loss: 0.7001
Epoch 7/100
35/35 [==============================] - 16s 467ms/step - loss: 0.7534 - val_loss: 0.6768
Epoch 8/100
35/35 [==============================] - 16s 464ms/step - loss: 0.7308 - val_loss: 0.6859
Epoch 9/100
35/35 [==============================] - 17s 480ms/step - loss: 0.7077 - val_loss: 0.6841
Epoch 10/100
35/35 [==============================] - 17s 472ms/step - loss: 0.700

Nice! Much better than what we had previously! Keep optimizing and the performance will keep improving.
Worth trying: stemming and lemmatization. This is something I'm skipping for now.

In the Kaggle world, to get a top score you should have an ensemble of models. Let's check a little bit of ensembling!


## Ensembling

Few months back I made a simple ensembler but I didn't have time to develop it fully. It can be found here: https://github.com/abhishekkrthakur/pysembler . I'm going to use some part of it here:

- 앙상블 학습은 여러 개의 모델을 결합하여 하나의 모델보다 더 좋은 성능을 내는 머신러닝 기법이다. 앙상블 학습의 핵심은 여러 개의 약 분류기 (Weak Classifier)를 결합하여 강 분류기(Strong Classifier)를 만드는 것이다. 그리하여 모델의 정확성이 향상된다.

In [100]:
# this is the main ensembling class. how to use it is in the next cell!
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
import os
import sys
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format="[%(asctime)s] %(levelname)s %(message)s",
    datefmt="%H:%M:%S", stream=sys.stdout)
logger = logging.getLogger(__name__)


class Ensembler(object):
    def __init__(self, model_dict, num_folds=3, task_type='classification', optimize=roc_auc_score,
                 lower_is_better=False, save_path=None):
        """
        Ensembler init function
        :param model_dict: model dictionary, see README for its format
        :param num_folds: the number of folds for ensembling
        :param task_type: classification or regression
        :param optimize: the function to optimize for, e.g. AUC, logloss, etc. Must have two arguments y_test and y_pred
        :param lower_is_better: is lower value of optimization function better or higher
        :param save_path: path to which model pickles will be dumped to along with generated predictions, or None
        """

        self.model_dict = model_dict
        self.levels = len(self.model_dict)
        self.num_folds = num_folds
        self.task_type = task_type
        self.optimize = optimize
        self.lower_is_better = lower_is_better
        self.save_path = save_path

        self.training_data = None
        self.test_data = None
        self.y = None
        self.lbl_enc = None
        self.y_enc = None
        self.train_prediction_dict = None
        self.test_prediction_dict = None
        self.num_classes = None

    def fit(self, training_data, y, lentrain):
        """
        :param training_data: training data in tabular format
        :param y: binary, multi-class or regression
        :return: chain of models to be used in prediction
        """

        self.training_data = training_data
        self.y = y

        if self.task_type == 'classification':
            self.num_classes = len(np.unique(self.y))
            logger.info("Found %d classes", self.num_classes)
            self.lbl_enc = LabelEncoder()
            self.y_enc = self.lbl_enc.fit_transform(self.y)
            kf = StratifiedKFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, self.num_classes)
        else:
            self.num_classes = -1
            self.y_enc = self.y
            kf = KFold(n_splits=self.num_folds)
            train_prediction_shape = (lentrain, 1)

        self.train_prediction_dict = {}
        for level in range(self.levels):
            self.train_prediction_dict[level] = np.zeros((train_prediction_shape[0],
                                                          train_prediction_shape[1] * len(self.model_dict[level])))

        for level in range(self.levels):

            if level == 0:
                temp_train = self.training_data
            else:
                temp_train = self.train_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):
                validation_scores = []
                foldnum = 1
                for train_index, valid_index in kf.split(self.train_prediction_dict[0], self.y_enc):
                    logger.info("Training Level %d Fold # %d. Model # %d", level, foldnum, model_num)

                    if level != 0:
                        l_training_data = temp_train[train_index]
                        l_validation_data = temp_train[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])
                    else:
                        l0_training_data = temp_train[0][model_num]
                        if type(l0_training_data) == list:
                            l_training_data = [x[train_index] for x in l0_training_data]
                            l_validation_data = [x[valid_index] for x in l0_training_data]
                        else:
                            l_training_data = l0_training_data[train_index]
                            l_validation_data = l0_training_data[valid_index]
                        model.fit(l_training_data, self.y_enc[train_index])

                    logger.info("Predicting Level %d. Fold # %d. Model # %d", level, foldnum, model_num)

                    if self.task_type == 'classification':
                        temp_train_predictions = model.predict_proba(l_validation_data)
                        self.train_prediction_dict[level][valid_index,
                        (model_num * self.num_classes):(model_num * self.num_classes) +
                                                       self.num_classes] = temp_train_predictions

                    else:
                        temp_train_predictions = model.predict(l_validation_data)
                        self.train_prediction_dict[level][valid_index, model_num] = temp_train_predictions
                    validation_score = self.optimize(self.y_enc[valid_index], temp_train_predictions)
                    validation_scores.append(validation_score)
                    logger.info("Level %d. Fold # %d. Model # %d. Validation Score = %f", level, foldnum, model_num,
                                validation_score)
                    foldnum += 1
                avg_score = np.mean(validation_scores)
                std_score = np.std(validation_scores)
                logger.info("Level %d. Model # %d. Mean Score = %f. Std Dev = %f", level, model_num,
                            avg_score, std_score)

            logger.info("Saving predictions for level # %d", level)
            train_predictions_df = pd.DataFrame(self.train_prediction_dict[level])
            train_predictions_df.to_csv(os.path.join(self.save_path, "train_predictions_level_" + str(level) + ".csv"),
                                        index=False, header=None)

        return self.train_prediction_dict

    def predict(self, test_data, lentest):
        self.test_data = test_data
        if self.task_type == 'classification':
            test_prediction_shape = (lentest, self.num_classes)
        else:
            test_prediction_shape = (lentest, 1)

        self.test_prediction_dict = {}
        for level in range(self.levels):
            self.test_prediction_dict[level] = np.zeros((test_prediction_shape[0],
                                                         test_prediction_shape[1] * len(self.model_dict[level])))
        self.test_data = test_data
        for level in range(self.levels):
            if level == 0:
                temp_train = self.training_data
                temp_test = self.test_data
            else:
                temp_train = self.train_prediction_dict[level - 1]
                temp_test = self.test_prediction_dict[level - 1]

            for model_num, model in enumerate(self.model_dict[level]):

                logger.info("Training Fulldata Level %d. Model # %d", level, model_num)
                if level == 0:
                    model.fit(temp_train[0][model_num], self.y_enc)
                else:
                    model.fit(temp_train, self.y_enc)

                logger.info("Predicting Test Level %d. Model # %d", level, model_num)

                if self.task_type == 'classification':
                    if level == 0:
                        temp_test_predictions = model.predict_proba(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict_proba(temp_test)
                    self.test_prediction_dict[level][:, (model_num * self.num_classes): (model_num * self.num_classes) +
                                                                                        self.num_classes] = temp_test_predictions

                else:
                    if level == 0:
                        temp_test_predictions = model.predict(temp_test[0][model_num])
                    else:
                        temp_test_predictions = model.predict(temp_test)
                    self.test_prediction_dict[level][:, model_num] = temp_test_predictions

            test_predictions_df = pd.DataFrame(self.test_prediction_dict[level])
            test_predictions_df.to_csv(os.path.join(self.save_path, "test_predictions_level_" + str(level) + ".csv"),
                                       index=False, header=None)

        return self.test_prediction_dict


In [101]:
# specify the data to be used for every level of ensembling:
train_data_dict = {0: [xtrain_tfv, xtrain_ctv, xtrain_tfv, xtrain_ctv], 1: [xtrain_glove]}
test_data_dict = {0: [xvalid_tfv, xvalid_ctv, xvalid_tfv, xvalid_ctv], 1: [xvalid_glove]}

model_dict = {0: [LogisticRegression(), LogisticRegression(), MultinomialNB(alpha=0.1), MultinomialNB()],

              1: [xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)]}

ens = Ensembler(model_dict=model_dict, num_folds=3, task_type='classification',
                optimize=multiclass_logloss, lower_is_better=True, save_path='')

ens.fit(train_data_dict, ytrain, lentrain=xtrain_glove.shape[0])
preds = ens.predict(test_data_dict, lentest=xvalid_glove.shape[0])

[19:12:30] INFO Found 3 classes
[19:12:30] INFO Training Level 0 Fold # 1. Model # 0
[19:12:32] INFO Predicting Level 0. Fold # 1. Model # 0
[19:12:32] INFO Level 0. Fold # 1. Model # 0. Validation Score = 0.626621
[19:12:32] INFO Training Level 0 Fold # 2. Model # 0
[19:12:34] INFO Predicting Level 0. Fold # 2. Model # 0
[19:12:34] INFO Level 0. Fold # 2. Model # 0. Validation Score = 0.616463
[19:12:34] INFO Training Level 0 Fold # 3. Model # 0
[19:12:37] INFO Predicting Level 0. Fold # 3. Model # 0
[19:12:37] INFO Level 0. Fold # 3. Model # 0. Validation Score = 0.619626
[19:12:37] INFO Level 0. Model # 0. Mean Score = 0.620903. Std Dev = 0.004244
[19:12:37] INFO Training Level 0 Fold # 1. Model # 1
[19:12:57] INFO Predicting Level 0. Fold # 1. Model # 1
[19:12:57] INFO Level 0. Fold # 1. Model # 1. Validation Score = 0.573485
[19:12:57] INFO Training Level 0 Fold # 2. Model # 1
[19:13:17] INFO Predicting Level 0. Fold # 2. Model # 1
[19:13:17] INFO Level 0. Fold # 2. Model # 1. Val

In [102]:
# check error:
multiclass_logloss(yvalid, preds[1])

0.4612325034912492

Thus, we see that ensembling improves the score by a great extent! Since this is supposed to be a tutorial only I wont be providing any CSVs that you can submit to the leaderboard.